In [ ]:
import imutils
import itertools
import cv2
import colorsys
import numpy as np
from framedir import FrameDir
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 8]

In [ ]:
def show(*args):
    if len(args) > 1:
        imgs = np.hstack(tuple(args))
        imgs = cv2.cvtColor(imgs, cv2.COLOR_BGR2RGB, imgs)
    else:
        imgs = cv2.cvtColor(args[0], cv2.COLOR_BGR2RGB)
    plt.imshow(imgs)
    del imgs
    plt.show()

In [ ]:
# ffmpeg -i /var/tmp/cam1.mp4 '%04d.jpg'
fd = FrameDir('/var/tmp/faux200327.announce')
frames = list(fd.load(f) for f in range(320, 840))

In [ ]:
blurred = list(cv2.GaussianBlur(frame, (17, 17), 0) for frame in frames)

In [ ]:
def median(frame_range, rate = 15):
    (s, e) = frame_range
    return np.median(np.array(blurred[s:e:rate]), axis=0).astype('uint8')
show(median((150, 300)))

In [ ]:
def mask(middle, now):
    im = cv2.absdiff(middle, now)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY, im)
    im = im.astype(np.float32) / 255.
    return im

    
def mask_rgb(m):
    return np.dstack((m,m,m))


def show_mask(m):
    show(mask_rgb(m))


def time_mask(mid, frame_range, fudges = 5, skips = 2):
    (s, e) = frame_range
    im = mask(mid, blurred[s])
    for n in blurred[s+skips : e : skips]:
        x = mask(mid, n)
        im = cv2.add(im, x, im)
        del x
    return im


def thresh_mask(im, scale):
    _, t = cv2.threshold(im / scale, 1, 1, cv2.THRESH_TRUNC)
    return t

In [ ]:
def mask_around(when):
    median_range = 300
    med = median((max(when - median_range // 2, 0), min(when + median_range // 2, len(frames))))
    return time_mask(med, (when, when+30))

def mask_scaled(im):
    return im / np.max(im)

#cv2.sumElems(mask_around(300))[0] / 1e3
#np.max(mask_around(500))

#t = time_mask(median((200, 400)), (300, 330))
show(mask_rgb(mask_scaled(mask_around(200))), mask_rgb(mask_scaled(mask_around(280))), mask_rgb(mask_scaled(mask_scaled(mask_around(200)) * mask_scaled(mask_around(300)))))

In [ ]:
src = frames[310]
im = src.astype(np.float32) / 255.
t_rgb = mask_rgb(t)
i_rgb = mask_rgb(1 - t)
cut = cv2.multiply(im, t_rgb, im, 1, cv2.CV_32F)
back = cv2.multiply(frames[220].astype(np.float32) / 255., i_rgb, dtype=cv2.CV_32F)
show(cv2.add(cut, back, cut))